# Worksheet 08

Name: shivacharan oruganti
UID: U55255882

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.893348785127524, 3.6484110154602454, 5.726443710886897, 5.323520034640589, 5.861118906065801, 4.643320615759754, 3.7846244260792705, 4.53012269603122, 7.044049530917386, 5.046140739723119]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.660620189363492, 8.734775393929056, 6.825281450524653, 8.6180475208078, 8.789018921250214, 8.715535557172668, 7.149764548456055, 8.603659001327703, 7.370023297694853, 7.894520795590322]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.046140739723119, 7.894520795590322, 7.370023297694853, 8.603659001327703, 7.044049530917386, 4.53012269603122, 3.7846244260792705, 7.149764548456055, 8.715535557172668, 4.643320615759754]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean1 = 5, variance1 = 1
mean2 = 8, variance2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [6]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2) / len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x: (x - mean[1]) ** 2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[5.046140739723119, 4.53012269603122, 3.7846244260792705, 4.643320615759754]
[7.894520795590322, 7.370023297694853, 8.603659001327703, 7.044049530917386, 7.149764548456055, 8.715535557172668]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.501052119398341,  mean_2 = 7.796258788526498
var_1 = 0.2078689168111898,  var_2 = 0.44534509439296505


The KMeans-derived means are somewhat close to the true values, but the variances are notably smaller than the true variances. This discrepancy can be attributed to the nature of KMeans clustering, which tends to produce spherical clusters, leading to underestimation of the true variances in this case.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [8]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.046140739723119
probability of observing that point if it came from cluster 0 =  0.06165238434732642
probability of observing that point if it came from cluster 1 =  4.6943814744257995e-09
point =  7.894520795590322
probability of observing that point if it came from cluster 0 =  2.581461824643557e-58
probability of observing that point if it came from cluster 1 =  0.8742628004361115
point =  7.370023297694853
probability of observing that point if it came from cluster 0 =  8.291620977923407e-42
probability of observing that point if it came from cluster 1 =  0.5666328619192424
point =  8.603659001327703
probability of observing that point if it came from cluster 0 =  4.98650596102308e-85
probability of observing that point if it came from cluster 1 =  0.17317117646699065
point =  7.044049530917386
probability of observing that point if it came from cluster 0 =  6.0861326403308e-33
probability of observing that point if it came from cluster 1 =  0.21513826489089477
point =  

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [11]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),  sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([prob_s0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_s0_x),
    sum([prob_s1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_s1_x)
]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.501052103833718,  mean_2 = 7.796258736169246
var_1 = 0.2078689142633293,  var_2 = 0.4453452299060321


The updated values are relatively close to the initial KMeans estimates. The EM algorithm, of which this is one step, refines initial guesses (like those from KMeans) to better fit the data's distribution.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [14]:
prob_s0_x_updated = []  # P(S_0 | X_i)
prob_s1_x_updated = []  # P(S_1 | X_i)

for p in data:
    # P(X_i | S_j) for both clusters
    pdf_s0 = norm.pdf(p, mean[0], np.sqrt(var[0]))
    pdf_s1 = norm.pdf(p, mean[1], np.sqrt(var[1]))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_val = prob_c[0] * pdf_s0 + prob_c[1] * pdf_s1

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x_updated.append(pdf_s0 * prob_c[0] / prob_x_val)
    prob_s1_x_updated.append(pdf_s1 * prob_c[1] / prob_x_val)

updated_probs_without_helper = list(zip(data, prob_s0_x_updated, prob_s1_x_updated))
updated_probs_without_helper

[(5.046140739723119, 0.9995703687243499, 0.0004296312756500334),
 (7.894520795590322, 9.213718975596359e-13, 0.9999999999990786),
 (7.370023297694853, 3.016869652869186e-09, 0.9999999969831304),
 (8.603659001327703, 5.303491200391891e-18, 1.0),
 (7.044049530917386, 3.234188260726116e-07, 0.999999676581174),
 (4.53012269603122, 0.9999935427405459, 6.457259454203441e-06),
 (3.7846244260792705, 0.999999949895251, 5.010474905846276e-08),
 (7.149764548456055, 7.316922597606976e-08, 0.999999926830774),
 (8.715535557172668, 7.026391121356325e-19, 1.0),
 (4.643320615759754, 0.9999847031130641, 1.5296886935879454e-05)]

Overall, while there are some adjustments in the posterior probabilities due to the updated parameters, the general pattern of cluster assignments remains consistent. This suggests that the updated parameters provide a slightly better fit to the data, refining the probabilities.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [15]:
hard_assignments = [0 if prob_s0_x_val > prob_s1_x_val else 1 for prob_s0_x_val, prob_s1_x_val in zip(prob_s0_x_updated, prob_s1_x_updated)]

hard_assigned_data = list(zip(data, hard_assignments))
hard_assigned_data

[(5.046140739723119, 0),
 (7.894520795590322, 1),
 (7.370023297694853, 1),
 (8.603659001327703, 1),
 (7.044049530917386, 1),
 (4.53012269603122, 0),
 (3.7846244260792705, 0),
 (7.149764548456055, 1),
 (8.715535557172668, 1),
 (4.643320615759754, 0)]